In [1]:
import torch
import numpy as np

data = torch.load("data/nasbench101/all_nasbench101.pt")  # 按你的路径替换
# data = torch.load("data/nasbench201/all_nasbench201.pt")

def is_undirected(adj):
    a = np.asarray(adj)
    return np.array_equal(a, a.T)

undirected = 0
directed = 0
first_directed = None

for k, v in data.items():
    adj = v["adj"]
    if is_undirected(adj):
        undirected += 1
    else:
        directed += 1
        if first_directed is None:
            first_directed = (k, np.sum(np.asarray(adj) != np.asarray(adj).T))

print("undirected:", undirected, "directed:", directed)
print("first_directed:", first_directed)

/tmp/ipykernel_138858/3529554614.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("data/nasbench101/all_nasbench101.pt")  # 按你的路径替换


undirected: 0 directed: 423624
first_directed: (0, 18)


In [5]:
import torch
import numpy as np
from nnformer.data_process.nasbench.dataset import NasbenchDataset

dataset = NasbenchDataset(
    logger=type("L", (), {"info": print})(),
    dataset="nasbench101",
    part="train",
    data_path="data/nasbench101/all_nasbench101.pt",
    percent=100,
    lbd_consistency=0,
)
sample = dataset[0]
adj = sample["code_adj"].cpu().numpy()
print("code_adj_undirected:", np.array_equal(adj, adj.T))

Building dataset train from .pth file
Loading cached data from ./data/nasbench101/nasbench101_0/train_100_cached_data.pth
Finish Loading dataset train; Number of samples: 100; Time: 0.00s
code_adj_undirected: False


/home/rtx4090/code/python/current/GraphRegression/nnformer/data_process/nasbench/dataset.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(cache_file)


In [20]:
from nnformer.data_process.fixed_length_sampler import FixedLengthBatchSampler
from torch.utils.data import DataLoader  # 正确导入 PyTorch 的 DataLoader

trainset = NasbenchDataset(
    logger=type("L", (), {"info": print})(),
    dataset="nasbench101",
    part="train",
    data_path="data/nasbench101/all_nasbench101.pt",
    percent=100,
    lbd_consistency=0,
)
train_sampler = FixedLengthBatchSampler(
    trainset, "nasbench101", 32, include_partial=True
)
train_loader = DataLoader(
    trainset,
    shuffle=(train_sampler is None),
    num_workers=1,
    pin_memory=True,
    persistent_workers=True,
    batch_sampler=train_sampler,
)

Building dataset train from .pth file
Loading cached data from ./data/nasbench101/nasbench101_0/train_100_cached_data.pth
Finish Loading dataset train; Number of samples: 100; Time: 0.00s


In [28]:
for batch in train_loader:
    print(len(batch))
    break  # 仅打印第一个batch，避免输出过多

8
